In [4]:
# imports

import socket
import sys
import os
import math
import time
import threading

In [5]:
# sender GBN

In [12]:
# cmd line parameters
DEBUG_MODE = False
R_ADDRESS = "127.0.0.1"
R_PORT_NUMBER = 12365
OTHER_PORT = 11166
PACKET_LENGTH = 32
PACKET_GEN_RATE = 200
MAX_PACKETS = 100
WINDOW_SIZE = 3
MAX_BUFFER_SIZE = 10

# getting input
args = len(sys.argv)
i = 1
while(i < args):
    if(sys.argv[i] == "-d"):
        DEBUG_MODE = True
    if(sys.argv[i] == "-s"):
        i += 1
        R_ADDRESS = int(sys.argv[i])
    if(sys.argv[i] == "-p"):
        i += 1
        R_PORT_NUMBER = int(sys.argv[i])
    if(sys.argv[i] == "-l"):
        i += 1
        PACKET_LENGTH = int(sys.argv[i])
    if(sys.argv[i] == "-r"):
        i += 1
        PACKET_GEN_RATE = int(sys.argv[i])
    if(sys.argv[i] == "-n"):
        i += 1
        MAX_PACKETS = int(sys.argv[i])
    if(sys.argv[i] == "-w"):
        i += 1
        WINDOW_SIZE = int(sys.argv[i])
    if(sys.argv[i] == "-b"):
        i += 1
        MAX_BUFFER_SIZE = int(sys.argv[i])
    i += 1

In [7]:
# packet generator

def gen_packs():
    
    period = 1/PACKET_GEN_RATE
    start_time = time.time()
    global create_id
    global buffer
    
    while (True):
        if(len(buffer) < MAX_BUFFER_SIZE):
            st = str(create_id)+"~" + "x"*PACKET_LENGTH
            # lock.acquire()
            buffer.append(bytes(st,'utf-8'))
            # lock.release()
            print("created : "+str(create_id))
            create_id += 1
            # print(time.time() - start_time)
            time.sleep(period)
            
        if(retrans > 8 or succ_send >= MAX_PACKETS):
            break

In [8]:
# packet transmitter

def send_packs():
    
    global un_acks
    global sent_count
    global timers
    
    while (True):
        lock.acquire()
        if(un_acks < WINDOW_SIZE and len(buffer) > un_acks):
            sock.sendto(buffer[un_acks], (R_ADDRESS, R_PORT_NUMBER))
            sent_count += 1
            word = buffer[un_acks].decode('utf-8')
            un_acks += 1
            timers.append(time.time())
            print("sent : " + word[:word.find('~')] + " " + str(un_acks))
        lock.release()

        if(retrans > 8 or succ_send >= MAX_PACKETS):
            break

In [9]:
# packet reciever

def recv_acks():
    
    global un_acks
    global timers
    global succ_send
    global retrans
    global RTT_avg
    global tot_time
    global next_pack_id
    
    while (True):
        
        data, addr = sock_ack.recvfrom(1024)

        if(data != None):
            val = int(data.decode('utf-8'))
            if(val == -1):
                break
            if(val == next_pack_id):
                while(True):
                    if(len(timers) > 0):
                        break
                tot_time += (time.time() - timers[0])
                del timers[0]
                print("success : " + str(succ_send))
                succ_send += 1
                RTT_avg = tot_time/succ_send
                retrans = 0
                next_pack_id += 1
                lock.acquire()
                del buffer[0]
                un_acks -= 1
                lock.release()

        if(retrans > 8 or succ_send >= MAX_PACKETS):
            break

In [17]:
#data

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock_ack = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock_ack.bind((R_ADDRESS, OTHER_PORT))

buffer = []
create_id = 0

un_acks = 0
next_pack_id = 0
timers = []
sent_count = 0

retrans = 0
succ_send = 0
tot_time = 0
RTT_avg = 0


# process

t1 = threading.Thread(target = gen_packs,args = ())
t2 = threading.Thread(target = send_packs,args = ())
t3 = threading.Thread(target = recv_acks,args = ())

lock = threading.Lock()

t1.start()
t2.start()
t3.start()


while(True):

    lock.acquire()
    if(len(timers) > 0):
        if(succ_send > 10):
            if((time.time() - timers[0]) > 2*RTT_avg):
                timers.clear()
                un_acks = 0
                retrans += 1
        else:
            if((time.time() - timers[0]) > 0.1):
                timers.clear()
                un_acks = 0
                retrans += 1
    lock.release()
    
    if(retrans > 10 or succ_send >= MAX_PACKETS):
        break

t1.join()
t2.join()
t3.join()

print("Sender Terminated")

created : 0
sent : 0 1
success : 0
created : 1
sent : 1 1success : 1

created : 2
sent : 2 1success : 2

created : 3
sent : 3 1
created : 4
sent : 4 2
created : 5
sent : 5 3
created : 6
created : 7
created : 8
created : 9
created : 10
created : 11
created : 12
sent : 3 1
sent : 4 2
sent : 5 3
success : 3
success : 4sent : 6 3

success : 5
sent : 7 3
created : 13
success : 6sent : 8 3

success : 7sent : 9 3

sent : 10 3
created : 14
created : 15
created : 16
created : 17
sent : 8 1
sent : 9 2
sent : 10 3
sent : 8 1success : 8

sent : 9 2
sent : 10 3
success : 9
created : 18sent : 11 3

sent : 12 3success : 10

success : 11sent : 13 3

success : 12
sent : 14 3
success : 13sent : 15 3

sent : 16 3
sent : 14 1
sent : 15 2
sent : 16 3
created : 19
sent : 14 1
sent : 15 2
sent : 16 3
success : 14
sent : 17 3
sent : 15 1
sent : 16 2
sent : 17 3
success : 15
sent : 18 3
created : 20
sent : 16 1
sent : 17 2
sent : 18 3
sent : 16 1
sent : 17 2
success : 16sent : 18 3

created : 21
success : 17
s

In [18]:
RTT_avg

0.000636754035949707

In [ ]:
sent_count/succ_send

In [ ]:
st = "x"*12
print(st)

In [ ]:
time.time()

In [ ]:
# Workup

In [ ]:
UDP_IP = "127.0.0.1"
UDP_PORT = 5005
MESSAGE = b"Hello, World!"

print ("UDP target IP:", UDP_IP)
print ("UDP target port:", UDP_PORT)
print ("message:", MESSAGE)

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

In [ ]:
print(MESSAGE.decode("utf-8"))

In [ ]:
rList = []



arr = bytes(b"Hello, World")
print(arr)

In [ ]:
sys.getsizeof(arr)

In [ ]:
bh = [1,2,3,4]
del bh[0]
print(bh[0])
len(bh)

In [ ]:
word = "12~jjlisd"
word[:word.find('~')]